In [1]:
import pandas as pd
from openff.toolkit import Molecule
from openfe import SmallMoleculeComponent
from rdkit import Chem
from rdkit.Chem import AllChem

# read the csv file path
set_1_csv_path = "./dataset/FXR/FEP_set_1.csv"
set_2_csv_path = "./dataset/FXR/FEP_set_2.csv"

ligand_df = pd.read_csv(set_1_csv_path, sep=',', header=0)
print(ligand_df.head())

ligand_mols = []
for index, row in ligand_df.iterrows():
    ligand_id = row['Ligand_ID']
    smiles_string = row['SMILES']
    # ligand_mol = Molecule.from_smiles(smiles_string)
    # ligand_mol.generate_conformers(n_conformers=1)
    ligand_mol = Chem.MolFromSmiles(smiles_string)
    ligand_mol_h = Chem.AddHs(ligand_mol)
    AllChem.EmbedMolecule(ligand_mol_h, randomSeed=7)
    small_ligand_mol = SmallMoleculeComponent.from_rdkit(ligand_mol_h, ligand_id)
    ligand_mols.append(small_ligand_mol)


  Ligand_ID                                             SMILES
0    FXR_17  CCOC(=O)c1ccc(NC(=O)c2c3CN(CCc3nn2c4ccccc4)S(=...
1    FXR_45  CCOC(=O)c1ccc(NC(=O)c2c3CN(CCc3nn2c4ccc(OC(F)(...
2    FXR_46  NC(=O)c1ccc(NC(=O)c2c3CN(CCc3nn2c4ccccc4)S(=O)...
3    FXR_47  CCOC(=O)c1cccc(NC(=O)c2c3CN(CCc3nn2c4ccccc4)S(...
4    FXR_48  CCOC(=O)Cc1ccc(NC(=O)c2c3CN(CCc3nn2c4ccccc4)S(...


In [2]:
print("name: ", ligand_mols[0].name)

name:  FXR_17


In [3]:
from openfe.protocols.openmm_utils.omm_settings import OpenFFPartialChargeSettings
from openfe.protocols.openmm_utils.charge_generation import bulk_assign_partial_charges

charge_settings = OpenFFPartialChargeSettings(partial_charge_method="am1bcc", off_toolkit_backend="ambertools")

charged_ligands = bulk_assign_partial_charges(
    molecules=ligand_mols,
    overwrite=False,
    method=charge_settings.partial_charge_method,
    toolkit_backend=charge_settings.off_toolkit_backend,
    generate_n_conformers=charge_settings.number_of_conformers,
    nagl_model=charge_settings.nagl_model,
    processors=1
)

Generating charges:   7%|█▌                      | 1/15 [01:06<15:27, 66.25s/it]/home/craigv-z/micromamba/envs/openfe/bin/wrapped_progs/antechamber: Fatal Error!
Cannot properly run "/home/craigv-z/micromamba/envs/openfe/bin/sqm -O -i sqm.in -o sqm.out".
Generating charges:   7%|█▌                      | 1/15 [01:13<17:14, 73.87s/it]


KeyboardInterrupt: 

In [ ]:
import openfe
mapper = openfe.LomapAtomMapper(max3d=1.0, element_change=False)
scorer = openfe.lomap_scorers.default_lomap_score
network_planner = openfe.ligand_network_planning.generate_minimal_spanning_network

ligand_network = network_planner(
    ligands=charged_ligands,
    mappers=[mapper],
    scorer=scorer
)

In [ ]:
from openfe.utils.atommapping_network_plotting import plot_atommapping_network
plot_atommapping_network(ligand_network)



In [ ]:
# 制作原子图

from openfe.setup import LomapAtomMapper
mapper = LomapAtomMapper()
lomap_mapping = next(mapper.suggest_mappings(ligand_mols[0], ligand_mols[10]))

In [ ]:
from kartograf import KartografAtomMapper
# Build Kartograf Atom Mapper
mapper = KartografAtomMapper(atom_map_hydrogens=True)


In [ ]:

# Get Mapping
kartograf_mapping = next(mapper.suggest_mappings(ligand_mols[0], ligand_mols[10]))



In [ ]:
# We can display the atom mapping in 2D by calling it
kartograf_mapping

In [ ]:
import openfe
from openfe.setup.ligand_network_planning import generate_minimal_spanning_network

# Create an MST network
mst_network = generate_minimal_spanning_network(
    ligands=ligand_mols,
    scorer=openfe.lomap_scorers.default_lomap_score,
    mappers=[KartografAtomMapper(atom_map_hydrogens=True),])


In [ ]:
# Visualize the MST network
from openfe.utils.atommapping_network_plotting import plot_atommapping_network

plot_atommapping_network(mst_network)

